In [1]:
%matplotlib inline

In [5]:
import numpy as np
import lmfit
from scipy.io import readsav

In [11]:
from DSTPolarimeterLib import MuellerMatrixMirror, MuellerMatrixRotation

In [14]:
#--- dtor * deg = rad
dtor = 0.017453292519943295

---

In [62]:
def residual(par, mmHSP, mm_ir, sigma):
    
    mmHSP_model = model(par, mm_ir)
    
    return (mmHSP_model - mmHSP) / sigma

def model(par, mm_ir):
    
    mm_45 = MuellerMatrixMirror(par["mir_delta"].value*dtor, par["mir_p"].value, gen=True)
    mm_45 /= mm_45[0,0]
    
    # depolarization
    mm_45[1,1] -= par["mir_depol"].value
    # QU component
    mm_45[1,2] += par["mir_QU"].value
    mm_45[2,1] += par["mir_QU"].value
    # difference between [0,1] and [1,0]
    mm_45[0,1] += par["mir_pConst"].value
    mm_45[1,0] -= par["mir_pConst"].value
    
    # 以下两行只改变了delta的符号，因为其余符号被反的位置数值上都是0
    mm_45[3,:] = -mm_45[3,:]
    mm_45[:,3] = -mm_45[:,3]
    
    #
    mat = mm_45 @ MuellerMatrixRotation(par["th_dst2mm45"].value*dtor)
    mat = MuellerMatrixRotation(par["th_mm452mmir"].value*dtor) @ mat
    mat = MuellerMatrixRotation(par["th_mmir2hsp"].value*dtor) @ mm_ir @ mat
    
    sc = par["sc"].value
    mat = np.array([
        [1+sc, 0, 0, 0],
        [0   , 1, 0, 0],
        [0   , 0, 1, 0],
        [0   , 0, 0, 1]
    ]) @ mat
    
    mat /= mat[0,0]
    
    return mat

---

fitted Mueller Matrix of HSP

In [4]:
mmHSP = np.array([
    [1., -0.03454526, -0.02421685, 0.02406610],
    [0.02891437, -0.23698555, -0.95226598, 0.04610067],
    [0.02636235, -0.59941919, 0.14209622, 0.68590858],
    [0.01450438, -0.72102226, 0.12382938, -0.65461389]
], dtype=np.float64)

Mueller Matrix of image rotator

In [29]:
wave = 8600
imgrot = np.deg2rad(-59.462886810302734)
npzFile = np.load("/nwork/kouui/dstsp/data/calibration/save/mmirFilter.npz")
posWave = np.argmin( abs(npzFile["wlFilter"]*10-wave) )  # nm to angstrom
angleArray = np.deg2rad( npzFile["angleArray"] )
posAngle = np.argmin( abs(angleArray-imgrot) )

print("posWave : {}, posAngle : {}".format(posWave, posAngle))

dangle = imgrot - angleArray[posAngle]
mm_ir = MuellerMatrixRotation(dangle) @ npzFile["mmsFilter"][posWave,posAngle,:,:] @ MuellerMatrixRotation(-dangle)

posWave : 812, posAngle : 77


In [30]:
del npzFile

Mueller Matrix of mirror

In [15]:
# we try different model for mm_45

fitting parameters

In [68]:
par = lmfit.Parameters()
par.add( "th_dst2mm45", value=0, min=-180, max=180, vary=True)
par.add( "th_mm452mmir", value=90, min=-180, max=180, vary=True)
par.add( "th_mmir2hsp", value=0, min=-180, max=180, vary=True)
par.add( "sc", value=0.0, min=0, max=0.2, vary=True)
par.add( "mir_p", value= -0.0304, min=-1, max=1, vary=True)
par.add( "mir_delta", value=30.447, min=-180, max=180, vary=True)  # deg
par.add( "mir_depol", value=0.0, min=0, max=1, vary=False)
par.add( "mir_QU", value=0.0, min=-0.5, max=0.5, vary=False)
par.add( "mir_pConst", value=0.0, min=-0.2, max=0.2, vary=False)

fitting error

In [51]:
sigma = np.ones((4,4))*0.001

start fitting

In [69]:
result = lmfit.minimize(residual, par, args=(mmHSP, mm_ir, sigma),method='leastsq', ftol=1E-12, xtol=1E-12)
lmfit.report_fit(result)

[[Fit Statistics]]
    # function evals   = 143
    # data points      = 16
    # variables        = 6
    chi-square         = 27678.133
    reduced chi-square = 2767.813
    Akaike info crit   = 131.293
    Bayesian info crit = 135.928
[[Variables]]
    th_dst2mm45:    7.21105220 +/- 0        (0.00%) (init= 0)
    th_mm452mmir:   95.2831177 +/- 0        (0.00%) (init= 90)
    th_mmir2hsp:   -39.8977229 +/- 0        (0.00%) (init= 0)
    sc:             0          +/- 0        (nan%) (init= 0)
    mir_p:         -0.02289207 +/- 0        (0.00%) (init=-0.0304)
    mir_delta:      16.6011343 +/- 0        (0.00%) (init= 30.447)
    mir_depol:      0 (fixed)
    mir_QU:         0 (fixed)
    mir_pConst:     0 (fixed)


/home/kouui/virtualenvs/python35/lib/python3.5/site-packages/lmfit/printfuncs.py:140: RuntimeWarning: invalid value encountered in double_scalars
  spercent = '({0:.2%})'.format(abs(par.stderr/par.value))


origin

In [70]:
result.residual.reshape(4,4) * sigma

array([[ 0.        ,  0.00448847, -0.00474118, -0.00934518],
       [-0.00095146,  0.0108648 , -0.07314988, -0.00840923],
       [ 0.01021624, -0.09519374,  0.05202496,  0.08647849],
       [ 0.00430975, -0.0427016 ,  0.02161341, -0.01863115]])

with `depol`

In [53]:
result.residual.reshape(4,4) * sigma

array([[ 0.        ,  0.01598738,  0.01115057, -0.02189748],
       [-0.01174474, -0.03593874, -0.04146636, -0.00955769],
       [-0.002117  , -0.04135916,  0.00539744,  0.090765  ],
       [ 0.00506408,  0.01677387, -0.03305   , -0.02247056]])

with `depol` and `QU`

In [67]:
result.residual.reshape(4,4) * sigma

array([[ 0.        ,  0.01787846,  0.02420255,  0.00117319],
       [-0.03766389, -0.00865181, -0.07173503,  0.00513126],
       [ 0.00404433, -0.02063702,  0.01196712,  0.08476418],
       [-0.00824356,  0.03449218, -0.00021339, -0.0260496 ]])

with `depol`, `QU`, `pConst`

In [61]:
result.residual.reshape(4,4) * sigma

array([[ 0.        ,  0.01040839,  0.02357508,  0.00017655],
       [-0.03324022, -0.01648263, -0.06986656,  0.01187595],
       [ 0.00479372,  0.00021814, -0.00469058,  0.05787638],
       [-0.00849887,  0.00291082,  0.00377151, -0.05119571]])